In [ ]:
import gdown
file_id = '1CSfABalJZpKpwSUcGHSWzCplfmdHREms'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'konli.zip', quiet=False)
!unzip konli.zip

Downloading...
From (original): https://drive.google.com/uc?id=1CSfABalJZpKpwSUcGHSWzCplfmdHREms
From (redirected): https://drive.google.com/uc?id=1CSfABalJZpKpwSUcGHSWzCplfmdHREms&confirm=t&uuid=fdae6245-2d94-4a21-9b81-171654d319d7
To: /content/konli.zip
100%|██████████| 41.7M/41.7M [00:00<00:00, 60.5MB/s]


Archive:  konli.zip
  inflating: xnli.dev.ko.tsv         
  inflating: snli_1.0_train.kor.tsv  
replace multinli.train.ko.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: multinli.train.ko.tsv   
  inflating: xnli.test.ko.tsv        


In [ ]:
# 필요라이브러리
import pandas as pd

In [ ]:
TRAIN_MULTINLI_DF = pd.read_csv('multinli.train.ko.tsv', sep='\t',on_bad_lines ='skip')
print(TRAIN_MULTINLI_DF['gold_label'].unique())
# entailment : 첫번째 문장이 두번째 문장을 의미
# contradiction : 첫번재 문장이 두번째 문장을 의미하자 않는 것
# neutral : 첫번째 문장이 두번째 문장을 의미하거나 반대하지 않는경우
TRAIN_MULTINLI_DF.tail()

['neutral' 'entailment' 'contradiction' nan]


,sentence1,sentence2,gold_label
385489,"분명히, 캘리포니아는 더 잘 할 수 있고, 더 잘해야 한다.",캘리포니아는 더 잘할 수 없다.,contradiction
385490,"한때 유럽에서 가장 아름다운 거리로 여겨졌는데, 이는 원래의 많은 건물들이 교체되었...",그래서 원래의 많은 건물들이 편의점으로 대체되었다.,neutral
385491,하우스보트는 영국 라지의 전성기의 아름답게 보존된 전통이다.,하우스보트의 전통은 영국 라지가 여전히 강해지는 동안 시작되었다.,entailment
385492,사망 기사는 그의 평론가의 신디케이트 TV 쇼에서 동료 검토 자 Roger Eber...,부고문은 아름다웠고 연예계에서의 그의 업적에 대해 현물로 쓰여졌다.,neutral
385493,"내가 해야 한다는 걸 알거나, 아니면 누가 하라고 하는 것보다 그녀를 밀고하는 것에...",남편이 요즘 너무 과로해서 이 근처에서 많은 일을 부탁할 용기가 나지 않는다.,neutral


In [ ]:
TRAIN_SNLI_DF = pd.read_csv('snli_1.0_train.kor.tsv', sep='\t',on_bad_lines ='skip')
TRAIN_SNLI_DF.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [ ]:
TRAIN_XNLI_DF = pd.read_csv('xnli.dev.ko.tsv', sep='\t',on_bad_lines ='skip')
TRAIN_XNLI_DF.head()


,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


- 사용데이터셋 : KoNLI
- 모델 : BERT 모델 기반 자연어 추론 분류기 구축
- 자연어 문장을 주면...'entailment', 'contradiction','neutral' 중 하나로 분류
- 데이터 로딩, 전처리, 토큰화, 모델정의학습, 평가, 추론

- 데이터셋 : SNLI + XNLI 데이터를 결합하고 결측치 제거
- Bert모델에 사용할 BartTokenizer 로드
- 두 문장을 입력으로 받아서 Bert모델에 적합한 형태로 변환 작업
  - 함수구현
- 학습과 평가
- 베스트모델 로드
- 테스트데이터 평가
- 추론

In [ ]:
import tensorflow as tf

In [ ]:
# 데이터 합침 TRAIN_MULTINLI_DF, TRAIN_SNLI_DF
TRAIN_MULTINLI_SNLI_DF =  pd.concat([TRAIN_MULTINLI_DF, TRAIN_SNLI_DF]).reset_index(drop=True)
TRAIN_MULTINLI_SNLI_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935646 entries, 0 to 935645
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   sentence1   935646 non-null  object
 1   sentence2   935600 non-null  object
 2   gold_label  935583 non-null  object
dtypes: object(3)
memory usage: 21.4+ MB


In [42]:
# BERT 로드
from transformers import BertTokenizer, TFBertForSequenceClassification
# BERT 토크나이져 로드 전이학습
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',cashe_dir='bert_ckpt',do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [43]:
# MAX_LEN 모델의 입력길이 제한값과 데이터의 평균적인 길이를 고려해서 작성  BERT 최대가 512토큰
# 1. 데이터셋에서 각 문장 상의 토큰길이를 계산해서 분포를 확인  데이터의 95%가 특정 길이 이하로 나타나다. --> 해당 길이를 max_len
import numpy as np
token_len = []
for sent1, sent2 in zip(TRAIN_MULTINLI_SNLI_DF['sentence1'], TRAIN_MULTINLI_SNLI_DF['sentence2']):
  token = tokenizer.encode(sent1, sent2,add_special_tokens=True)
  token_len.append(len(token))

# 히스토그램으로 분포 확인
import matplotlib.pyplot as plt
plt.hist(token_len, bins=50)
plt.show()

# 95번째 백분위수에 해당하는 값
MAX_LEN = np.percentile(token_len,95)
print(f"95% 데이터의 토큰 길이 이하 : {MAX_LEN}")

# 70~90% 를 사용하기도 함

Token indices sequence length is longer than the specified maximum sequence length for this model (2247 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [71]:
# 하이퍼 파라메터 설정
BATCH_SIZE = 32
NUM_EPOCHS = 3
MAX_LEN = 36*2  # 최대 문장의 길이 (평균길이의 두배)

In [72]:
TRAIN_MULTINLI_SNLI_DF.dropna(inplace=True)
TRAIN_MULTINLI_SNLI_DF.drop_duplicates(inplace=True)
TRAIN_MULTINLI_SNLI_DF.reset_index(drop=True, inplace=True)
TRAIN_MULTINLI_SNLI_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934547 entries, 0 to 934546
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   sentence1   934547 non-null  object
 1   sentence2   934547 non-null  object
 2   gold_label  934547 non-null  object
dtypes: object(3)
memory usage: 21.4+ MB


In [73]:
# 데이터 크기 확인
TEST_XNLI_DF = pd.read_csv('xnli.test.ko.tsv', sep='\t',on_bad_lines ='skip')
print(TRAIN_MULTINLI_SNLI_DF.shape, TEST_XNLI_DF.shape)

(934547, 3) (4954, 3)


In [83]:
import warnings
warnings.filterwarnings(action='ignore')

In [84]:
# 두개의 문장을 토큰화하는 함수 정의
def bert_tokenizer(sent1, sent2, MAX_LEN):
  encoded_dict = tokenizer.encode_plus(
      text = sent1,         # 첫번째
      text_pair = sent2,    # 두번째
      add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
      max_length = MAX_LEN,  # Pad & truncate all sentences.
      padding = True,  # Padding strategy
      return_attention_mask = True
      )
  return encoded_dict['input_ids'], encoded_dict['attention_mask'], encoded_dict['token_type_ids']

In [ ]:
input_ids, attention_masks,token_type_ids = [],[],[]  # 문장 토큰화, 토큰이 패딩인지 단어인지(0, 1),
                                                      # 문장구분을 위해서 첫번째 문장은 0, 두번째문장은1
from tqdm import tqdm
for sent1,sent2 in  tqdm(zip(TRAIN_MULTINLI_SNLI_DF['sentence1'], TRAIN_MULTINLI_SNLI_DF['sentence2']),total = len(TRAIN_MULTINLI_SNLI_DF)):
  try:
    input_id, attention_mask, token_type_id = bert_tokenizer(sent1,sent2,MAX_LEN)
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
  except Exception as e:
    print(e)
    print(sent1, sent2)

# 넘파이 배열로 변환
input_ids_list = np.array(input_ids,dtype=int)
attention_masks_list = np.array(attention_masks,dtype=int)
token_type_ids_list = np.array(token_type_ids,dtype=int)
train_snli_xnli_inputs = (input_ids_list,attention_masks_list,token_type_ids_list)

  8%|▊         | 70273/934547 [00:57<08:31, 1688.89it/s]